# 1) Titolo e obiettivi

Lezione 26: Anomaly Detection - Trovare l'ago nel pagliaio con metodi non supervisionati

---

## Mappa della lezione

| Sezione | Contenuto | Tempo stimato |
|---------|-----------|---------------|
| 1 | Titolo, obiettivi, tassonomia anomalie | 5 min |
| 2 | Teoria profonda: IF, LOF, soglie, contamination | 20 min |
| 3 | Schema mentale: workflow detection | 5 min |
| 4 | Demo: IF base, LOF locale, tuning, frodi, ensemble | 30 min |
| 5 | Esercizi guidati + debug | 15 min |
| 6 | Conclusione operativa | 10 min |
| 7 | Checklist di fine lezione + glossario | 5 min |
| 8 | Changelog didattico | 2 min |

---

## Obiettivi della lezione

Al termine di questa lezione sarai in grado di:

| # | Obiettivo | Verifica |
|---|-----------|----------|
| 1 | Distinguere **anomalie globali, locali, contestuali** | Sai descrivere un esempio per ciascuna? |
| 2 | Applicare **Isolation Forest** | Sai usare contamination e decision_function? |
| 3 | Applicare **Local Outlier Factor** | Sai scegliere n_neighbors? |
| 4 | Scegliere **soglie e contamination** con criterio | Sai usare percentili e F1 per tuning? |
| 5 | Valutare con **precision/recall/F1** | Sai interpretare falsi positivi vs recall? |
| 6 | Combinare **ensemble IF + LOF** | Sai fare AND/OR tra predizioni? |

---

## L'idea centrale: cosa sono le anomalie

```
DISTRIBUZIONE NORMALE:                 ANOMALIE:

    ●●●●●●●●●●●                       Globale: ★ (lontano da tutto)
   ●●●●●●●●●●●●●                      
  ●●●●●●●●●●●●●●●                     Locale: ○ (in zona meno densa)
   ●●●●●●●●●●●●●                      
    ●●●●●●●●●●●          ★            Contestuale: punto normale, 
       cluster                          ma anomalo a Natale
```

**Anomaly detection:** trovare i punti che "non appartengono" al pattern normale.

---

## Perché è difficile

| Sfida | Descrizione | Conseguenza |
|-------|-------------|-------------|
| **Classi sbilanciate** | 0.1%-5% di anomalie | Accuracy inutile |
| **Assenza di etichette** | Spesso non sai cosa è anomalo | Non puoi fare supervised |
| **Soglia da scegliere** | Dove metti il cutoff? | Trade-off precision/recall |
| **Dati eterogenei** | Scale diverse, feature miste | Scaling obbligatorio |

---

## I 2 algoritmi chiave: Isolation Forest vs LOF

```
ISOLATION FOREST:                      LOCAL OUTLIER FACTOR:

Domanda: "Quanti tagli                Domanda: "La mia zona è
servono per isolare                   meno densa di quella
questo punto?"                        dei miei vicini?"

    ┌───────────────┐                    ●●●●  densità alta
    │   ● ●   ★     │                    ● ○   densità bassa
    │ ●   ●         │                    ↑
    └───────────────┘                    outlier locale!
         ↑                               
   Anomalia isolata                   LOF > 1 → anomalo
   con 2 split!                       LOF ≈ 1 → normale
```

| Aspetto | Isolation Forest | LOF |
|---------|------------------|-----|
| **Tipo anomalie** | Globali, sparse | Locali, in cluster densi |
| **Scalabilità** | Ottima (O(n log n)) | Peggiore (O(n² k)) |
| **Output** | decision_function (score) | negative_outlier_factor_ |
| **Quando preferire** | Tante feature, anomalie lontane | Poche feature, anomalie in mezzo |

---

## Prerequisiti

| Concetto | Dove lo trovi | Verifica |
|----------|---------------|----------|
| StandardScaler | Lezione 13, 20 | Sai perché scalare prima di IF/LOF? |
| Alberi decisionali | Lezione 14 | Capisci come IF fa split casuali? |
| k-NN concetto | Lezione base | Sai cosa sono i k vicini? |
| Precision/Recall/F1 | Lezione 17 | Sai interpretare trade-off? |

**Cosa useremo:** make_blobs, StandardScaler, IsolationForest, LocalOutlierFactor, classification_report.

# 2) Teoria concettuale
## 2.1 Cosa sono le anomalie e perche' sono difficili
- Un'anomalia e' un'osservazione che si discosta nettamente dal comportamento atteso.
- Tipi: globali (lontane da tutto), locali (in zone meno dense), contestuali (anomale solo in certi contesti/tempi).
- Sfide: classi sbilanciate, assenza di etichette, soglie da impostare, dati spesso non scalati e con feature eterogenee.
## 2.2 Algoritmi chiave
- Isolation Forest: isola i punti con tagli casuali; anomalie richiedono meno tagli. Input matrice (n_samples, n_features) scalata; output etichette (+1 regolare, -1 anomalo) e decision_function (score). Errori tipici: contamination incoerente, dati non scalati, troppi estimators lenti.
- Local Outlier Factor (LOF): confronta densita' locale con quella dei vicini. Input matrice (n_samples, n_features), output etichette (-1 anomalo) e negative_outlier_factor_. Errori tipici: n_neighbors troppo alto/basso, dati non scalati.
## 2.3 Metriche e soglie
- Con etichette: precision, recall, F1, confusion matrix. Con punteggi continui: PR curve, ROC, percentili come soglia.
- Contamination: stima della % di anomalie. Se troppo alta genera falsi positivi; se troppo bassa perde anomalie.
- Regola pratica: partire da 1-5% e fare tuning empirico con F1/recall in base alle priorita' business.


## 2.2 Quando usare IF o LOF
- Preferisci Isolation Forest se le anomalie sono sparse e globali, e vuoi un modello che scala bene con molte feature.
- Preferisci LOF se le anomalie sono locali (zone meno dense dentro cluster) e hai dati 2D/3D o pochi attributi dopo PCA.
- Se non conosci il tipo di anomalia, prova entrambi e confronta F1/recall in base al costo degli errori.


## 2.3 Soglie, percentili e interpretazione degli score
- Isolation Forest: `decision_function` restituisce score (alto = normale). Scegli soglie con percentili (es. 95-esimo) o usa `contamination` per fissare la percentuale di anomalie.
- LOF: `negative_outlier_factor_` (piu' negativo = piu' anomalo). Puoi ordinare gli score e scegliere un cutoff.
- Interpretazione: confronta sempre score/soglie con esempi noti di anomalie per validare che il modello non stia marcando punti plausibili come outlier.


# 3) Schema mentale / mappa decisionale
Workflow: load -> check/clean -> scale -> scegli modello (IF/LOF) -> stima contamination/soglia -> valuta -> interpreta -> iterazioni.
Decision map sintetica:
1. Scala le feature numeriche (StandardScaler/RobustScaler).
2. Se hai label di anomalie, usa PR/F1 per scegliere soglia o contamination.
3. Prova sia Isolation Forest (globali) sia LOF (locali); confronta.
4. Se anomalie sono poche e vicine a cluster densi, LOF e' spesso migliore; se sono sparse globali, Isolation Forest.
5. Documenta la soglia scelta e verifica stabilita' su run multipli.
Micro-checklist: nessun NaN, forme coerenti, contamination compatibile con la % attesa, almeno alcuni punti marcati come anomali.


# 4) Sezione dimostrativa
Panoramica demo:
- Demo 1: Isolation Forest su dataset sintetico, metriche e grafico.
- Demo 2: LOF su anomalie locali vs globali.
- Demo 3: Tuning della contamination con F1.
- Demo 4: Simulazione frodi con Isolation Forest.
- Demo 5: Ensemble IF + LOF e confronto strategie.


## Demo 1 - Isolation Forest base
Perche': mostrare isolamento rapido di anomalie globali su dati scalati.
Metodi: `StandardScaler` (input n_samples x n_features, output scalato), `IsolationForest` (etichette +1/-1, decision_function). Checkpoint: nessun NaN, contamination coerente, almeno alcune anomalie rilevate.


In [ ]:
# Demo 1: Isolation Forest su dataset sintetico
# Scopo: generare dati con poche anomalie globali, scalare le feature, applicare Isolation Forest e valutare precision/recall/F1.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support, classification_report, confusion_matrix
from sklearn.datasets import make_blobs

np.random.seed(42)
plt.close('all')

# 1) Crea dataset con anomalie
X_normal, _ = make_blobs(n_samples=800, centers=3, cluster_std=0.6, random_state=42)
anomalies = np.random.uniform(low=-6, high=6, size=(40, 2))
X = np.vstack([X_normal, anomalies])
y_true = np.hstack([np.zeros(len(X_normal)), np.ones(len(anomalies))]).astype(int)
print(f"Forma dati: {X.shape}, anomalie attese: {y_true.sum()}")
assert X.shape[0] == y_true.shape[0], "Shape incoerente"
assert not np.isnan(X).any(), "NaN nei dati"

# 2) Scala le feature
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print(f"Forma dopo scaling: {X_scaled.shape}")

# 3) Isolation Forest con contamination ipotizzata al 5%
iso = IsolationForest(contamination=0.05, random_state=42)
y_pred = iso.fit_predict(X_scaled)
y_pred_bin = (y_pred == -1).astype(int)

# 4) Metriche
prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred_bin, average='binary', zero_division=0)
print(classification_report(y_true, y_pred_bin, digits=3))
print(f"Confusion matrix:{confusion_matrix(y_true, y_pred_bin)}")
print(f"Precision: {prec:.3f}, Recall: {rec:.3f}, F1: {f1:.3f}")
assert y_pred_bin.sum() > 0, "Nessuna anomalia rilevata"

# 5) Visualizzazione
fig, ax = plt.subplots(figsize=(6, 5))
ax.scatter(X_scaled[:, 0], X_scaled[:, 1], c=y_pred_bin, cmap='coolwarm', s=15, alpha=0.8)
ax.set_title('Isolation Forest: 0=normale, 1=anomalia (scalato)')
ax.set_xlabel('Feature 1 (scaled)')
ax.set_ylabel('Feature 2 (scaled)')
plt.tight_layout()
plt.show()


## Demo 2 - Local Outlier Factor
Perche': gestire anomalie locali che un modello globale potrebbe ignorare. Metodi: `LocalOutlierFactor` con n_neighbors e contamination. Checkpoint: almeno un'anomalia trovata, differenza tra anomalie globali e locali.


In [ ]:
# Demo 2: Local Outlier Factor (LOF)
# Scopo: evidenziare anomalie locali e globali con LOF su dati 2D scalati.
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import precision_recall_fscore_support

print("="*70)
print("DEMO 2 - Local Outlier Factor")
print("="*70)

np.random.seed(42)

# Dataset con cluster denso e sparso + anomalie
cluster_dense = np.random.randn(200, 2) * 0.3
cluster_sparse = np.random.randn(120, 2) * 2 + [5, 5]
local_anomalies = np.array([[2.5, 2.5], [2.0, 3.0], [3.0, 2.0], [-2, -2], [-2.5, -1.5]])
global_anomalies = np.array([[10, 10], [-8, 8]])
X_lof = np.vstack([cluster_dense, cluster_sparse, local_anomalies, global_anomalies])
y_true_lof = np.hstack([
    np.zeros(len(cluster_dense) + len(cluster_sparse)),
    np.ones(len(local_anomalies) + len(global_anomalies))
]).astype(int)

scaler_lof = StandardScaler()
X_lof_scaled = scaler_lof.fit_transform(X_lof)
print(f"Forma dataset LOF: {X_lof_scaled.shape}")
assert not np.isnan(X_lof_scaled).any(), "NaN nei dati LOF"

lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
y_pred_lof = lof.fit_predict(X_lof_scaled)
y_pred_lof_bin = (y_pred_lof == -1).astype(int)

prec, rec, f1, _ = precision_recall_fscore_support(y_true_lof, y_pred_lof_bin, average='binary', zero_division=0)
print(f"Precision: {prec:.3f}, Recall: {rec:.3f}, F1: {f1:.3f}")
assert y_pred_lof_bin.sum() > 0, "LOF non ha rilevato anomalie"

fig, ax = plt.subplots(figsize=(6, 5))
ax.scatter(X_lof_scaled[:, 0], X_lof_scaled[:, 1], c=y_pred_lof_bin, cmap='coolwarm', s=15, alpha=0.8)
ax.set_title('LOF: 0=normale, 1=anomalia (scalato)')
ax.set_xlabel('Feature 1 (scaled)')
ax.set_ylabel('Feature 2 (scaled)')
plt.tight_layout()
plt.show()


## Demo 3 - Tuning del parametro contamination
Perche': la % di anomalie attese impatta fortemente precision/recall. Strategia: grid manuale su contamination e confronto F1.


In [ ]:
# Demo 3: tuning del parametro contamination
# Scopo: mostrare l'impatto di contamination su precision/recall/F1 con Isolation Forest (dataset Demo 1).
contamination_values = [0.01, 0.02, 0.05, 0.10]

results_cont = []
for cont in contamination_values:
    iso_tune = IsolationForest(contamination=cont, random_state=42)
    pred = iso_tune.fit_predict(X_scaled)
    pred_bin = (pred == -1).astype(int)
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, pred_bin, average='binary', zero_division=0)
    results_cont.append({"contamination": cont, "precision": prec, "recall": rec, "f1": f1, "anomalie_predette": pred_bin.sum()})

res_df = pd.DataFrame(results_cont).sort_values(by='f1', ascending=False)
print(res_df)

best_row = res_df.iloc[0]
print(f"Miglior contamination per F1: {best_row['contamination']}, F1={best_row['f1']:.3f}")
assert best_row['anomalie_predette'] > 0, "Nessuna anomalia individuata nel tuning"


## Demo 4 - Anomaly detection simulazione frodi
Perche': applicare Isolation Forest a un dataset sbilanciato con feature eterogenee (importo, orario, distanza). Checkpoint: recall accettabile sulle frodi e controllo falsi positivi.


In [ ]:
# Demo 4: anomaly detection su simulazione frodi
# Scopo: simulare transazioni (2% frodi), scalare feature eterogenee e valutare Isolation Forest.
np.random.seed(42)

n_normal = 5000
n_fraud = 100  # 2% frodi

# Transazioni normali
normal_amount = np.abs(np.random.exponential(50, n_normal))
normal_hour = np.random.normal(14, 4, n_normal) % 24
normal_freq = np.random.poisson(5, n_normal)
normal_distance = np.abs(np.random.normal(20, 10, n_normal))

# Transazioni fraudolente
fraud_amount = np.abs(np.random.exponential(200, n_fraud))
fraud_hour = (np.random.normal(2, 3, n_fraud) % 24)
fraud_freq = np.random.poisson(1, n_fraud)
fraud_distance = np.abs(np.random.normal(200, 50, n_fraud))

X_fraud = np.vstack([
    np.column_stack([normal_amount, normal_hour, normal_freq, normal_distance]),
    np.column_stack([fraud_amount, fraud_hour, fraud_freq, fraud_distance])
])
y_fraud = np.hstack([np.zeros(n_normal), np.ones(n_fraud)]).astype(int)
print(f"Forma X_fraud: {X_fraud.shape}, frodi attese: {y_fraud.sum()}")
assert not np.isnan(X_fraud).any(), "NaN nel dataset frodi"

scaler_fraud = StandardScaler()
X_fraud_scaled = scaler_fraud.fit_transform(X_fraud)

iso_fraud = IsolationForest(contamination=0.02, random_state=42)
pred_fraud = iso_fraud.fit_predict(X_fraud_scaled)
pred_fraud_bin = (pred_fraud == -1).astype(int)

prec, rec, f1, _ = precision_recall_fscore_support(y_fraud, pred_fraud_bin, average='binary', zero_division=0)
print(classification_report(y_fraud, pred_fraud_bin, digits=3))
print(f"Precision: {prec:.3f}, Recall: {rec:.3f}, F1: {f1:.3f}")
assert pred_fraud_bin.sum() > 0, "Nessuna frode rilevata"


## Demo 5 - Ensemble IF + LOF
Perche': combinare punti di forza di modelli globali e locali. Strategie: AND (entrambi dicono anomalia) e OR (almeno uno). Valutiamo precision/recall/F1.


In [ ]:
# Demo 5: ensemble Isolation Forest + LOF
# Scopo: confrontare strategie AND/OR combinando IF e LOF sul dataset frodi.
lof_fraud = LocalOutlierFactor(n_neighbors=25, contamination=0.02)
# fit_predict restituisce -1 per anomalie
pred_lof = lof_fraud.fit_predict(X_fraud_scaled)
pred_lof_bin = (pred_lof == -1).astype(int)

# Strategie di combinazione
pred_and = ((pred_fraud_bin == 1) & (pred_lof_bin == 1)).astype(int)
pred_or = ((pred_fraud_bin == 1) | (pred_lof_bin == 1)).astype(int)

rows = []
for name, pred_vec in [("IsolationForest", pred_fraud_bin), ("LOF", pred_lof_bin), ("AND", pred_and), ("OR", pred_or)]:
    prec, rec, f1, _ = precision_recall_fscore_support(y_fraud, pred_vec, average='binary', zero_division=0)
    rows.append({"metodo": name, "precision": prec, "recall": rec, "f1": f1, "anomalie_predette": pred_vec.sum()})

res_ensemble = pd.DataFrame(rows).sort_values(by='f1', ascending=False)
print(res_ensemble)
assert len(res_ensemble) == 4, "Risultati ensemble mancanti"


# 5) Esercizi svolti (passo-passo)
## Esercizio 26.1 - Monitoraggio sensori industriali
Obiettivo: generare letture sensori, inserire anomalie e rilevarle con Isolation Forest, controllando recall e falsi positivi.


In [ ]:
# Esercizio 26.1: monitoraggio sensori industriali
# Passi: crea dati sensori, aggiungi anomalie, scala, applica Isolation Forest, valuta recall e falsi positivi.
np.random.seed(42)

n_normal_sensors = 10000
n_anomaly_sensors = 100

# Letture normali (macchinario funzionante)
temp_normal = np.random.normal(70, 5, n_normal_sensors)
pressure_normal = np.random.normal(100, 10, n_normal_sensors)
vibration_normal = np.random.normal(0.5, 0.1, n_normal_sensors)
rpm_normal = np.random.normal(3000, 100, n_normal_sensors)
energy_normal = np.random.normal(50, 5, n_normal_sensors)

# Anomalie (surriscaldamento, pressione anomala)
temp_anom = np.random.normal(95, 3, n_anomaly_sensors)
pressure_anom = np.random.normal(140, 5, n_anomaly_sensors)
vibration_anom = np.random.normal(1.0, 0.2, n_anomaly_sensors)
rpm_anom = np.random.normal(3200, 80, n_anomaly_sensors)
energy_anom = np.random.normal(70, 6, n_anomaly_sensors)

X_sensors = np.vstack([
    np.column_stack([temp_normal, pressure_normal, vibration_normal, rpm_normal, energy_normal]),
    np.column_stack([temp_anom, pressure_anom, vibration_anom, rpm_anom, energy_anom])
])
y_sensors = np.hstack([np.zeros(n_normal_sensors), np.ones(n_anomaly_sensors)]).astype(int)
print(f"Shape dati sensori: {X_sensors.shape}, anomalie attese: {y_sensors.sum()}")
assert not np.isnan(X_sensors).any(), "NaN nei dati sensori"

scaler_sensors = StandardScaler()
X_sensors_scaled = scaler_sensors.fit_transform(X_sensors)

iso_sensors = IsolationForest(contamination=0.01, random_state=42)
pred_sensors = iso_sensors.fit_predict(X_sensors_scaled)
pred_sensors_bin = (pred_sensors == -1).astype(int)

prec, rec, f1, _ = precision_recall_fscore_support(y_sensors, pred_sensors_bin, average='binary', zero_division=0)
print(f"Precision: {prec:.3f}, Recall: {rec:.3f}, F1: {f1:.3f}")
print(f"Anomalie rilevate: {pred_sensors_bin.sum()} su {y_sensors.sum()}")
assert rec > 0, "Recall nulla: alzare contamination"


## Esercizio 26.2 - Confronto LOF vs Isolation Forest
Obiettivo: su dati con anomalie globali e locali, confrontare F1 e scegliere l'algoritmo piu' adatto.


In [ ]:
# Esercizio 26.2: confronto LOF vs Isolation Forest
# Passi: dati con anomalie globali e locali, applica entrambi e confronta F1.
np.random.seed(42)

cluster_dense = np.random.randn(500, 2) * 0.5
cluster_sparse = np.random.randn(500, 2) * 1.5 + [6, 6]
global_anom = np.array([[15, 15], [-10, 10], [10, -10], [-10, -10], [0, 15]])
local_anom = np.array([[3, 3], [2.5, 3.5], [3.5, 2.5], [4, 4], [2, 4]])

X_comp = np.vstack([cluster_dense, cluster_sparse, global_anom, local_anom])
y_comp = np.hstack([np.zeros(len(cluster_dense) + len(cluster_sparse)), np.ones(len(global_anom) + len(local_anom))]).astype(int)
print(f"Shape dataset confronto: {X_comp.shape}")
assert not np.isnan(X_comp).any(), "NaN nei dati"

scaler_comp = StandardScaler()
X_comp_scaled = scaler_comp.fit_transform(X_comp)

iso_comp = IsolationForest(contamination=0.02, random_state=42)
pred_iso = (iso_comp.fit_predict(X_comp_scaled) == -1).astype(int)
lof_comp = LocalOutlierFactor(n_neighbors=30, contamination=0.02)
pred_lof = (lof_comp.fit_predict(X_comp_scaled) == -1).astype(int)

rows = []
for name, preds in [("IsolationForest", pred_iso), ("LOF", pred_lof)]:
    prec, rec, f1, _ = precision_recall_fscore_support(y_comp, preds, average='binary', zero_division=0)
    rows.append({"modello": name, "precision": prec, "recall": rec, "f1": f1, "anomalie_predette": preds.sum()})

res_comp = pd.DataFrame(rows)
print(res_comp)
assert len(res_comp) == 2, "Risultati mancanti"


## Esercizio 26.3 - Ottimizzazione della soglia
Obiettivo: usare gli anomaly score di Isolation Forest sulle frodi e scegliere la soglia migliore (percentile) per massimizzare F1.


In [ ]:
# Esercizio 26.3: ottimizzazione soglia su anomaly score
# Passi: usa gli score di Isolation Forest sul dataset frodi e seleziona il percentile che massimizza F1.
from numpy import percentile

scores = -iso_fraud.decision_function(X_fraud_scaled)  # piu' alto = piu' anomalo
print(f"Score: min {scores.min():.3f}, max {scores.max():.3f}, mean {scores.mean():.3f}")
assert scores.shape[0] == y_fraud.shape[0], "Shape inconsistente"

percentiles = [90, 92, 94, 95, 96, 97, 98]
rows = []
for p in percentiles:
    thr = percentile(scores, p)
    preds = (scores >= thr).astype(int)
    prec, rec, f1, _ = precision_recall_fscore_support(y_fraud, preds, average='binary', zero_division=0)
    rows.append({"percentile": p, "threshold": thr, "precision": prec, "recall": rec, "f1": f1, "anomalie_predette": preds.sum()})

res_thr = pd.DataFrame(rows).sort_values(by='f1', ascending=False)
print(res_thr)

best = res_thr.iloc[0]
print(f"Miglior soglia: percentile {best['percentile']} con F1={best['f1']:.3f}")
assert best['anomalie_predette'] > 0, "Soglia troppo alta, nessuna anomalia trovata"


# 6) Conclusione operativa

## 5 take-home messages

| # | Messaggio | Perché importante |
|---|-----------|-------------------|
| 1 | **IF per globali, LOF per locali** | Algoritmi diversi per pattern diversi |
| 2 | **Contamination = stima % anomalie** | Troppo alta → falsi positivi, troppo bassa → miss |
| 3 | **Sempre scalare prima** | IF e LOF sono sensibili alla scala |
| 4 | **Valuta con F1, non accuracy** | Classi sbilanciate rendono accuracy inutile |
| 5 | **Ensemble IF + LOF** | AND (conservativo) o OR (aggressivo) |

---

## Confronto sintetico: quando usare quale metodo

| Situazione | Metodo consigliato | Perché |
|------------|-------------------|--------|
| Anomalie sparse globali | Isolation Forest | Isola velocemente punti lontani |
| Anomalie locali in cluster | LOF | Confronta densità locale |
| Non sai il tipo | Entrambi + confronto F1 | Sicurezza empirica |
| Molte feature (>50) | IF (o IF su PCA) | LOF scala male |
| Serve interpretabilità | IF con decision_function | Score continuo intuitivo |

---

## Perché questi metodi funzionano

### 1) Isolation Forest - intuizione

```
Punto normale:                   Punto anomalo:

    ┌─────────────────┐             ┌─────────────────┐
    │ ┌───────────┐   │             │ ★               │
    │ │ ┌───────┐ │   │             └─────────────────┘
    │ │ │ ●     │ │   │                  1 split!
    │ │ └───────┘ │   │
    │ └───────────┘   │
    └─────────────────┘
         5 split...

Anomalia = pochi split per isolare → path corto nell'albero
```

### 2) LOF - intuizione

```
Densità punti A: 10 vicini in raggio 0.5 → alta
Densità punto B: 2 vicini in raggio 0.5 → bassa

LOF(B) = densità_media_vicini(B) / densità(B) > 1 → outlier!
```

---

## Metodi spiegati: reference card

| Metodo | Input | Output | Quando usarlo |
|--------|-------|--------|---------------|
| `IsolationForest(contamination)` | X scalato | labels (+1/-1), decision_function | Anomalie globali |
| `LocalOutlierFactor(n_neighbors)` | X scalato | labels (-1 anomalo), negative_outlier_factor_ | Anomalie locali |
| `decision_function(X)` | X nuovo | score (alto = normale) | Soglie custom |
| `fit_predict(X)` | X train | labels | Stesso set |
| `StandardScaler` | X raw | X scalato | Prima di IF/LOF |

---

## Errori comuni e debug rapido

| Errore | Perché sbagliato | Fix |
|--------|-----------------|-----|
| Non scalare | IF/LOF sensibili alla scala | StandardScaler prima |
| contamination = 0.5 | Metà dataset come anomalie? | Usa 0.01-0.05 realistici |
| n_neighbors = 1 in LOF | Troppo locale, instabile | Usa 20-50 |
| Valutare con accuracy | 99% accuracy con 1% anomalie è inutile | Usa F1, precision, recall |
| Ignorare false positive | Costo business! | Bilancia con recall |

---

## Template completo Anomaly Detection

```python
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import classification_report, f1_score
import numpy as np

# 1) Carica e scala
X = ...  # tua matrice
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2) Stima contamination (se hai label)
contamination_stima = y_true.mean() if 'y_true' in dir() else 0.05

# 3) Isolation Forest
iso = IsolationForest(contamination=contamination_stima, random_state=42)
labels_iso = iso.fit_predict(X_scaled)
labels_iso = (labels_iso == -1).astype(int)  # 1 = anomalo

# 4) LOF
lof = LocalOutlierFactor(n_neighbors=30, contamination=contamination_stima)
labels_lof = lof.fit_predict(X_scaled)
labels_lof = (labels_lof == -1).astype(int)  # 1 = anomalo

# 5) Ensemble (AND = conservativo, OR = aggressivo)
labels_and = labels_iso & labels_lof  # entrambi dicono anomalo
labels_or = labels_iso | labels_lof   # almeno uno dice anomalo

# 6) Valuta (se hai y_true)
if 'y_true' in dir():
    print("Isolation Forest:")
    print(classification_report(y_true, labels_iso))
    print("LOF:")
    print(classification_report(y_true, labels_lof))
    print("Ensemble AND:")
    print(f"F1: {f1_score(y_true, labels_and):.3f}")

# 7) Soglia custom con decision_function
scores = iso.decision_function(X_scaled)
threshold = np.percentile(scores, 5)  # 5% come anomalie
labels_custom = (scores < threshold).astype(int)
```

---

## Prossimi passi

| Argomento | Collegamento |
|-----------|--------------|
| One-Class SVM | Alternativa a IF per confini non-lineari |
| Autoencoder | Anomaly detection con reconstruction error |
| Time series | Anomalie temporali con rolling statistics |
| Domain-specific | Frodi, intrusioni, difetti industriali |

**Fine del corso unsupervised!** Hai ora tutti gli strumenti per clustering, riduzione e anomaly detection.

# 7) Checklist di fine lezione
- [ ] Ho scalato le feature prima di applicare IF/LOF.
- [ ] Ho scelto contamination o soglie in linea con la % attesa di anomalie.
- [ ] Ho verificato che almeno alcune anomalie vengano identificate.
- [ ] Ho confrontato precision/recall/F1 e documentato il trade-off.
- [ ] Ho testato almeno un modello globale (IF) e uno locale (LOF) se il contesto lo richiede.
- [ ] Ho salvato la soglia/scaler per riprodurre i risultati.

Glossario (termini usati):
- Anomalia/outlier: osservazione lontana dal comportamento atteso.
- Contamination: stima percentuale di anomalie usata dal modello.
- Isolation Forest: algoritmo basato su alberi casuali per isolare punti rari.
- Local Outlier Factor: algoritmo basato su densita' locale.
- n_neighbors: numero di vicini considerati da LOF.
- decision_function: score continuo di anomalia/normalita'.
- Precision: quota di predetti anomali che sono davvero anomali.
- Recall: quota di anomalie reali identificate.
- F1-score: media armonica precision/recall.
- Threshold: soglia sullo score per decidere anomalie.
- False positive: punto normale marcato come anomalo.


# 8) Changelog didattico

| Versione | Data | Modifiche |
|----------|------|-----------|
| 1.0 | 2024-01-20 | Creazione: IF e LOF base, metriche |
| 1.1 | 2024-01-28 | Aggiunto tuning contamination, demo frodi |
| 2.0 | 2024-02-05 | Integrata sezione ensemble IF + LOF |
| 2.1 | 2024-02-12 | Refactor con assert e checklist |
| **2.3** | **2024-12-19** | **ESPANSIONE COMPLETA:** mappa lezione 8 sezioni, tabella obiettivi, ASCII IF vs LOF, tassonomia anomalie (globali/locali/contestuali), confronto scalabilità, 5 take-home messages, template completo con ensemble AND/OR, soglia custom con percentili, reference card metodi, errori comuni |

---

## Note per lo studente

Questa lezione chiude il modulo **Unsupervised Learning**:

| Lezione | Argomento | Cosa hai imparato |
|---------|-----------|-------------------|
| 19 | Intro Unsupervised | Paradigma senza label |
| 20 | K-Means | Clustering a centroidi |
| 21 | Scelta K | Elbow, Silhouette, Gap |
| 22 | Gerarchico | Dendrogrammi e linkage |
| 23 | DBSCAN | Clustering a densità |
| 24 | PCA | Riduzione dimensionale |
| 25 | PCA + Clustering | Combinazione workflow |
| **26** | **Anomaly Detection** | **IF, LOF, soglie, ensemble** |

**Congratulazioni!** Hai completato il percorso dal clustering base fino all'anomaly detection.

**Prossimi passi consigliati:**
- Esercizi di consolidamento
- Progetti reali con dati tabulari
- Approfondimenti: UMAP, autoencoders, time series